<a href="https://colab.research.google.com/github/tulibraries/mads_mdx/blob/main/bookops_worldcat_rra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BookOps WorldCat
## Get MARC records with spreadsheet input

###1. Click on the "Files" icon to the left
### <---
Click the "Upload" icon and upload a .csv file containing ISBNs. Note which column contains the ISBN. Limit your .csv to 1000 records for best performance.

Once the file is uploaded, click on the vertical dot menu to copy its path

**PLEASE NOTE:** The uploaded file will only live in Google Colab for the length of this session. If you close this window and restart, you will need to re-upload.

### 2. Click on the arrow next to the code block below to install dependencies

This needs to be done every time you open a new session. When successful, a green checkmark appears next to the code block and you can proceed to the next step

In [ ]:
!pip install bookops-worldcat
!pip install pymarc

### 3. In the block below, tell the script the location of your search strings, then click the arrow to set these variables

- The `input_file` variable is the file path to your .csv (copied from step 1)
- The `isbn_col` variable is the column number where ISBNs are found in the .csv
   - Note: Column counts begin at '0' from left to right. The default is '6', which is the 7th column when you look visually at the .csv

In [ ]:
input_file = "/content/BookOps Test 3.csv"
isbn_col = 6

### 4. Click the arrow to search OCLC and write found records to a MARC file
If you want to change the default names for the output files, update the `output_file` and `not_found_file` variables below *before* running the code. Otherwise, run as is

The status of each search is output in the terminal below the code block



In [ ]:
output_file = "bookops_output.mrc" #update if desired
not_found_file = "not_found.csv"  # update if desired

import os
import csv
import time

from bookops_worldcat import MetadataSession, WorldcatAccessToken
from requests.exceptions import ReadTimeout
from pymarc import MARCWriter, Record
from google.colab import userdata

# Requires use of secret manager in colab 
WORLDCAT_KEY = userdata.get("WORLDCAT_KEY")
WORLDCAT_SECRET = userdata.get("WORLDCAT_SECRET")

token = WorldcatAccessToken(
    key=WORLDCAT_KEY,
    secret=WORLDCAT_SECRET,
    scopes="WorldCatMetadataAPI",
)

REQUEST_TIMEOUT = 30  # seconds
REQUEST_DELAY = 0.25  # seconds between rows

# Step 1. Initiate MetadataSession
with MetadataSession(authorization=token, timeout=REQUEST_TIMEOUT) as session:

    # Step 2. Open a CSV writer for rows not found
    with open(not_found_file, "w", newline="", encoding="utf-8") as nf:
        nf_writer = csv.writer(nf)
        nf_writer.writerow(["Title", "Author", "Editor", "Publisher", "Place_of_Publication", "Pub_Year", "ISBN"])  # optional header

        # Step 3. Read data from spreadsheet
        with open(input_file, "r", encoding="utf-8") as csvfile:
            reader = csv.reader(csvfile, delimiter=",")
            next(reader)  # skip header

            # Step 4. Iterate through each row in spreadsheet
            for row_num, row in enumerate(reader, start=1):
                isbn = row[isbn_col].strip()

                # Step 5. Query API using spreadsheet data
                brief_bib_response = session.brief_bibs_search(
                    q=f"bn:{isbn}",
                    inCatalogLanguage="eng",
                    itemType="book",
                    # itemSubType="book-digital",
                    # itemSubType="book-printbook"
                )

                # Step 6. Parse .json response
                brief_bib_json = brief_bib_response.json()

                if not brief_bib_json.get("briefRecords"):
                  print(f"[{row_num}] No record for ISBN {isbn}. Writing to {not_found_file}")
                  nf_writer.writerow(row)
                  continue

                matched_oclc_number = brief_bib_json["briefRecords"][0]["oclcNumber"]

                # Step 7: Retrieve full MARC record
                try:
                  get_response = session.bib_get(
                    matched_oclc_number,
                    responseFormat="application/marc"
                  )
                except ReadTimeout: # this doesn't seem to be working; will come back if timeout issue continues
                  print(f"[{row_num}] Timeout fetching OCLC no: {matched_oclc_number}")
                  nf_writer.writerow(row)
                  continue

                # Step 8: Build pymarc Record object
                pymarc_record = Record(get_response.content)

                # Step 9: Write MARC record to output file
                with open(output_file, "ab") as mrc_file:
                  writer = MARCWriter(mrc_file)
                  writer.write(pymarc_record)

                # Print to screen
                print(f"[{row_num}] Record found for {isbn}. Writing {matched_oclc_number} to {output_file}")

                time.sleep(REQUEST_DELAY)

### 5. The output MARC records can be downloaded from the Files section on the left.
### <---

Any records not found will be output into a not_found.csv file, which can also be found in the Files section on the left.

Again, the Files will only last for the duration of this session, so be sure to download them, as they will no longer be there the next time a session is initiated.